## EMNIST con MNIST features

En esta notebook se entrena una red, utilizando los features aprendidos de la base de datos del MNIST, para clasificar las clases de la base de datos de EMNIST, para ello se divide la red en la parte de feature layers de las capas convolucionales y la parte de clasificación de las capas densas. Se copian los pesos de las capas convolucionales aprendidos de MNIST y se entrenan los pesos de las capas de clasificación, obteniendo finalmente un accuracy de 93.68% para 15 épocas.

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
import imageio
from scipy.misc import imread, imsave, imresize
import PIL
from PIL import Image
from skimage import transform,io
from resizeimage import resizeimage
import scipy.io as sio
import scipy.misc
from skimage.transform import rescale, resize as rs, downscale_local_mean


/home/cuchuflito/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model=load_model('model.h5')


In [3]:
def impr(word):
    print ('\033[1m' + word + '\033[0m')
font = {'family': 'serif',
        'color':  'black',
        'weight': 15,
        'size': 16,
        }
file=sio.loadmat('emnist-letters.mat')


In [4]:
train=file['dataset'][0,0][0][0][0][0]
train_labels=file['dataset'][0,0][0][0][0][1]
test=file['dataset'][0,0][1][0][0][0]
test_labels=file['dataset'][0,0][1][0][0][1]
labels=[]
for i in train_labels:
    labels=np.append(labels,int(i[0]))

In [5]:
train_labels=labels-1   #empieza en 1 los labels----> les resto uno, sino hay problemas con el to_categorical 


In [6]:
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1) #tensorflow channels_last
num_classes = 26
feature_layers = [
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape = (28,28,1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    Flatten(),
]
classification_layers = [
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
]
for l in feature_layers:
    l.trainable = False

In [7]:
weights= [layer.get_weights() for layer in model.layers]
new_model= Sequential()

new_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
new_model.add(Conv2D(64, (3, 3), activation='relu'))
new_model.add(MaxPooling2D(pool_size=(2, 2)))
new_model.add(Dropout(0.25))
new_model.add(Flatten())
new_model.add(Dense(128, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(26, activation='softmax'))
for i in np.arange(0,len(feature_layers),1):
        new_model.layers[i].set_weights(weights[i])

In [8]:
new_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
train_labels = keras.utils.to_categorical(train_labels,26)

In [9]:
train = train.reshape(train.shape[0], 28, 28, 1).astype('float32')/255
new_model.fit(train,train_labels,batch_size=128*4, epochs=15, verbose=1)

Epoch 1/15
124800/124800 [==============================] - 299s 2ms/step - loss: 0.7552 - acc: 0.7730
Epoch 2/15
124800/124800 [==============================] - 284s 2ms/step - loss: 0.4104 - acc: 0.8703
Epoch 3/15
124800/124800 [==============================] - 314s 3ms/step - loss: 0.3466 - acc: 0.8893
Epoch 4/15
124800/124800 [==============================] - 300s 2ms/step - loss: 0.3109 - acc: 0.9009
Epoch 5/15
124800/124800 [==============================] - 298s 2ms/step - loss: 0.2862 - acc: 0.9071
Epoch 6/15
124800/124800 [==============================] - 286s 2ms/step - loss: 0.2675 - acc: 0.9133
Epoch 7/15
124800/124800 [==============================] - 286s 2ms/step - loss: 0.2538 - acc: 0.9158
Epoch 8/15
124800/124800 [==============================] - 291s 2ms/step - loss: 0.2409 - acc: 0.9197
Epoch 9/15
124800/124800 [==============================] - 288s 2ms/step - loss: 0.2331 - acc: 0.9226
Epoch 10/15
124800/124800 [==============================] - 329s 3ms/ste

In [10]:
#nmodel=load_model('modelo_EMNIST.h5')
new_model.save('modelo_EMNIST_ft_MNIST_.h5')

In [11]:
test = test.reshape(test.shape[0], img_rows, img_cols, 1).astype('float32')/255
prediction=new_model.predict_classes(test)

In [70]:
#print(prediction)
test_labels=file['dataset'][0,0][0][0][0]
#test_labels-=1  #corrijo porque empieza en 1
testlabels=[]
for i in file['dataset'][0,0][1][0][0][1]:
    testlabels=np.append(testlabels,int(i))
#print(file['dataset'][0,0][1][0][0][1])
testlabels-=1 #corrijo pq empieza en 1 y debe empezar en 0

#print(test.shape)
#for i in test_labels:
 #   testlabels=np.append(testlabels,int(i[0]))
#print(testlabels)
#print(prediction)
#print(np.where(prediction-testlabels==0)[0].shape[0])
impr('{:.2f}'.format(np.where(prediction-testlabels==0)[0].shape[0]*100/testlabels.shape[0])+'%')

93.68%
